In [ ]:
import dlib
import cv2
import numpy as np
import mediapipe as mp
from math import hypot

## Nose Filter

In [ ]:


cap = cv2.VideoCapture(0)
nose_image = cv2.imread("Resources/kittycat.png")
success, img = cap.read()
rows, cols, _ = img.shape
nose_mask = np.zeros((rows, cols), np.uint8)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("Landmarks/shape_predictor_68_face_landmarks.dat")

while True:
    success, img = cap.read()
    nose_mask.fill(0)
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(img)
    for face in faces:
        landmarks = predictor(imgGray, face)

        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_nose = (landmarks.part(31).x, landmarks.part(31).y)
        right_nose = (landmarks.part(35).x, landmarks.part(35).y)

        nose_width = int(hypot(left_nose[0] - right_nose[0],left_nose[1] - right_nose[1]) * 1.7)
        nose_height = int(nose_width * 0.77)

        top_left = (int(center_nose[0] - nose_width / 2),int(center_nose[1] - nose_height / 2))
        bottom_right = (int(center_nose[0] + nose_width / 2),int(center_nose[1] + nose_height / 2))

        nose_pig = cv2.resize(nose_image, (nose_width, nose_height))
        nose_pig_gray = cv2.cvtColor(nose_pig, cv2.COLOR_BGR2GRAY)
        _, nose_mask = cv2.threshold(nose_pig_gray, 25, 255, cv2.THRESH_BINARY_INV)

        nose_area = img[top_left[1]: top_left[1] + nose_height,top_left[0]: top_left[0] + nose_width]
        nose_no_area = cv2.bitwise_and(nose_area, nose_area, mask=nose_mask)

        img[top_left[1]: top_left[1] + nose_height, top_left[0]: top_left[0] + nose_width] = cv2.add(nose_no_area, nose_pig)

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

## Defrost

In [ ]:
def clean(img,imgSnow,pts):
    img_mask = np.zeros_like(img)
    cv2.circle(img_mask,(pts[0],pts[1]),pts[2],(255,255,255),-1)
    result = cv2.bitwise_and(img,img_mask)

    img_snow_mask = np.ones_like(imgSnow)
    img_snow_mask.fill(255)

    cv2.circle(img_snow_mask,(pts[0],pts[1]),pts[2],(0,0,0),-1)
    result_snow = cv2.bitwise_and(imgSnow,img_snow_mask)
    return cv2.bitwise_or(result,result_snow)

imgSnow = cv2.imread('Resources/snow.jpg')
imgSnow = cv2.resize(imgSnow,(640,480))
cap = cv2.VideoCapture(cv2.CAP_DSHOW)

mpHands = mp.solutions.hands 
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils
count = 0
points = []

while True:
    success,img = cap.read()
    img1 = img.copy()
    imgRGB = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    finalImg = cv2.addWeighted(img1,1,imgSnow,0.6,0)

    lmList = []
    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            for lm in handlandmark.landmark:
                h,w,_ = img1.shape
                lmList.append([int(lm.x*w),int(lm.y*h)])
            x,y,w,h = cv2.boundingRect(np.array(lmList))
            points.append([x + w // 2, y + h // 2,h // 2])
        count = 0
    else:
        if count<30:
            count+=1
        else:
            finalImg = cv2.addWeighted(img,1,imgSnow,0.6,0)
            points = []
            count=0

    if len(points)!=0:
        for pts in points:
            finalImg = clean(img,finalImg,pts)
    cv2.imshow('Image',finalImg)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()